# Профилирование и оптимизация выполнения кода

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/

## Задачи для совместного разбора

In [2]:
!pip install line_profiler
#pip install --user numpy==1.20

In [1]:
import numpy as np

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [4]:
A = np.random.randint(0, 1001, size=1_000_000) # shift tab

In [5]:
def slow(A):
    acc, cnt = 0, 0
    for a in A:
        b = a + 100
        acc += b
        cnt += 1
    return acc / cnt

In [6]:
slow(A)

600.549782

In [7]:
%%time # измеряет время выполнения всей ячейки
# на уровне ячейки
slow(A)
slow(A)
slow(A)

CPU times: total: 2.38 s
Wall time: 2.37 s


600.549782

In [8]:
%time slow(A) #измеряет время выполнения следующей строки.

CPU times: total: 797 ms
Wall time: 786 ms


600.549782

In [9]:
%%timeit
slow(A)

731 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def fast(A):
    cnt = len(A)
    s = sum(A) + 100*cnt
    return s / cnt

In [17]:
%%timeit
fast(A)

180 ms ± 491 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
(A + 100).mean()

6.61 ms ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [90]:
%load_ext line_profiler

In [91]:
%lprun

In [20]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.428601,0.077910,0.717509,0.772962,v
1,0.661402,-0.667595,-0.120428,0.841431,u


In [23]:
def select(df):
    mask = []
    for _, row in df.iterrows():
        if row["key"] in {"a", "b", "c", "d", "e"}:
            mask.append(True)
        else:
            mask.append(False)
    r = df[mask]
    return r

In [33]:
np.where(df['key'] == 'a')

(array([     42,      49,      52, ..., 1999942, 1999993, 1999995],
       dtype=int64),)

In [41]:
def select2(df):
    mask = df['key'].apply(lambda x: x in 'abcde')
    return df[mask]

In [42]:
%%time 
select2(df)

CPU times: total: 844 ms
Wall time: 852 ms


,col0,col1,col2,col3,key
24,-1.080934,0.569097,-0.195485,0.713653,e
25,2.794112,1.512965,0.165316,-0.345613,c
28,0.619905,-0.647973,0.411262,0.278362,e
31,-0.336526,1.344762,0.378538,-0.790413,c
42,-0.829648,0.606399,-0.385981,0.193759,a
...,...,...,...,...,...
1999979,0.375846,0.422224,-1.186499,0.119523,e
1999982,-2.827170,-0.044163,1.647108,0.258726,c
1999986,-0.425591,1.315511,0.790682,1.779919,e
1999993,-0.551422,0.506766,-0.814114,0.423465,a


In [25]:
%lprun -f select select(df.head(20_000))

## Лабораторная работа 1

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [1]:
import pandas as pd

In [2]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')

## Измерение времени выполнения кода

Создайте версию таблицы, содержащие строки строки для рецептов, которые были добавлены в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году. Полным описанием рецепта называется строка, полученная путем конкатенации названия и описания рецепта через пробел.

In [19]:
years = pd.DatetimeIndex(recipes['submitted']).year
rec10 = recipes[years == 2010]

In [23]:
rec10

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0
...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0


№1\.1 С использованием метода `DataFrame.iterrows` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вычисление полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [81]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    l = 0
    for _, row in df.iterrows():
        l += len(str(row['name'] + ' ' + row['description']))
    return l/len(df)      

In [29]:
get_mean_len_A(rec10)

265.501300390117

№1\.2. С использованием метода `DataFrame.apply` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вызываете метод apply у таблицы; в качестве аргумента передаете функцию, которая возвращает длину полного описания для каждой строки;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [75]:
def get_mean_len_B(df: pd.DataFrame) -> float:
    names = df[['name', 'description']].apply(lambda x: list(map(len, df['name'] + ' ' + 
                                                      df['description'])))['name']
    return names.mean()

In [76]:
get_mean_len_B(rec10)

265.501300390117

№1\.3. С использованием векторизованных методов серий `pd.Series`:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- при помощи векторизованной операции сложения получаете столбец с полным описанием;
    
- считаете длину каждого элемента столбца с полным описанием, воспользовавшись соответствующим строковым методом аксессора `.str`;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [46]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    lens = (rec10['name'] + ' ' + rec10['description']).str.len()
    return lens.mean()

In [48]:
get_mean_len_C(rec10)

265.501300390117

№1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [82]:
%time get_mean_len_A(rec10)

CPU times: total: 156 ms
Wall time: 144 ms


265.501300390117

In [83]:
%time get_mean_len_B(rec10)

CPU times: total: 0 ns
Wall time: 8.25 ms


265.501300390117

In [84]:
%time get_mean_len_C(rec10)

CPU times: total: 0 ns
Wall time: 0 ns


265.501300390117

In [85]:
%timeit get_mean_len_A(rec10)

161 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
%timeit get_mean_len_B(rec10)

8.08 ms ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [87]:
%timeit get_mean_len_C(rec10)

3.71 ms ± 72.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [16]:
import re
def get_word_reviews_count(df):
    word_reviews = {}
    for review_id, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word.lower() not in word_reviews:
                word_reviews[word.lower()] = set()
            word_reviews[word.lower()].add(review_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word.lower()] = len(word_reviews[word.lower()])
    return word_reviews_count

In [20]:
%%time 
get_word_reviews_count(reviews.head(5000))

CPU times: total: 672 ms
Wall time: 668 ms


{'last': 176,
 'week': 59,
 'whole': 199,
 'sides': 20,
 'of': 2414,
 'frozen': 125,
 'salmon': 33,
 'fillet': 8,
 'was': 2208,
 'on': 1132,
 'sale': 14,
 'in': 1711,
 'my': 1698,
 'local': 22,
 'supermarket': 4,
 'so': 1506,
 'i': 3940,
 'bought': 49,
 'tons': 7,
 'okay': 35,
 'only': 523,
 '': 3495,
 'but': 1456,
 'total': 24,
 'weight': 8,
 'over': 343,
 'pounds': 13,
 'this': 3241,
 'recipe': 2181,
 'is': 1591,
 'perfect': 371,
 'for': 2997,
 'even': 344,
 'though': 185,
 'it': 2899,
 'calls': 19,
 'steaks': 19,
 'cut': 248,
 'up': 567,
 'the': 3787,
 'into': 252,
 'individual': 10,
 'portions': 9,
 'and': 3778,
 'followed': 244,
 'instructions': 50,
 'exactly': 186,
 'im': 319,
 'one': 600,
 'those': 91,
 'food': 132,
 'combining': 2,
 'diets': 3,
 'left': 190,
 'out': 902,
 'white': 130,
 'wine': 67,
 'added': 743,
 'just': 934,
 'a': 3287,
 'dash': 20,
 'vinegar': 56,
 'instead': 416,
 'little': 562,
 'bit': 351,
 'not': 844,
 'enough': 187,
 'to': 2791,
 'change': 180,
 'taste'

№2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [9]:
%load_ext line_profiler

In [97]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews.head(5000))

Timer unit: 1e-07 s

Total time: 2.03224 s
File: C:\Users\cosit\AppData\Local\Temp\ipykernel_7864\1655901507.py
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def get_word_reviews_count(df):
     5         1          9.0      9.0      0.0      word_reviews = {}
     6      5000    6191942.0   1238.4     30.5      for review_id, row in df.dropna(subset=["review"]).iterrows():
     7      5000     865296.0    173.1      4.3          review = row["review"]
     8      5000     593883.0    118.8      2.9          words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
     9    264897     767115.0      2.9      3.8          for word in words:
    10    253856    1203254.0      4.7      5.9              if word.lower() not in word_reviews:
    11     11041      89240.0      8.1      0.4                  word_reviews[word.lower()] = set()
    12    264897    1840130.0      6.9      9.1              word_reviews[word.lower()].add(review_id)
    13         1          4.0      4.0      0.0      word_reviews_count = {}
    14      5000    4374621.0    874.9     21.5      for _, row in df.dropna(subset=["review"]).iterrows():
    15      5000     834890.0    167.0      4.1          review = row["review"]
    16      5000     575711.0    115.1      2.8          words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
    17    264897     786502.0      3.0      3.9          for word in words:
    18    264897    2199770.0      8.3     10.8              word_reviews_count[word.lower()] = len(word_reviews[word.lower()])
    19         1          3.0      3.0      0.0      return word_reviews_count

№2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [8]:
import string
from collections import Counter

In [13]:
def get_count2(df):
    def remove_pun(text):
        #for pun in string.punctuation + '1234567890':
            #text = text.replace(pun, '')
        text = re.sub(r'[^a-zA-Zа-яЁА-ЯЁ ]', '', text)
        text = text.lower()
        return ' '.join(list(set(text.split(' '))))
    return Counter(" ".join(df["review"].apply(remove_pun)).split())

In [14]:
get_count2(reviews.head(5000))

Counter({'white': 130,
         'steaks': 19,
         'a': 3287,
         'last': 176,
         'left': 190,
         'bit': 351,
         'to': 2791,
         'supermarket': 4,
         'dish': 334,
         'though': 185,
         'sale': 14,
         'lunch': 96,
         'individual': 10,
         'even': 345,
         'lucky': 3,
         'out': 902,
         'followed': 244,
         'instructions': 50,
         'enough': 187,
         'vinegar': 56,
         'today': 86,
         'calls': 19,
         'wine': 67,
         'portions': 9,
         'taste': 380,
         'one': 601,
         'up': 569,
         'change': 180,
         'im': 322,
         'over': 343,
         'the': 3788,
         'food': 132,
         'little': 562,
         'salmon': 33,
         'only': 524,
         'on': 1132,
         'super': 121,
         'in': 1711,
         'week': 59,
         'into': 252,
         'combining': 2,
         'cut': 248,
         'for': 2997,
         'whole': 199,
       

In [11]:
df = reviews
import re
def remove_pun(text):
        #for pun in string.punctuation + '1234567890':
            #text = text.replace(pun, '')
        text = re.sub(r'[^a-zA-Zа-яЁА-ЯЁ ]', '', text)
        text = text.lower()
        return ' '.join(list(set(text.split(' '))))
" ".join(df["review"].apply(remove_pun)).split()

AttributeError: 'StringMethods' object has no attribute 'apply'

In [115]:
%%time
get_count2(reviews.head(5000))

CPU times: total: 109 ms
Wall time: 120 ms


Counter({'just': 934,
         'exactly': 186,
         'change': 180,
         'though': 185,
         'im': 322,
         'salmon': 33,
         'so': 1507,
         'my': 1699,
         'tons': 7,
         'pounds': 13,
         'cut': 248,
         'it': 2899,
         'frozen': 125,
         'instead': 416,
         'lunch': 96,
         'local': 22,
         'but': 1457,
         'this': 3243,
         'left': 190,
         'white': 130,
         'sale': 14,
         'perfect': 371,
         'not': 844,
         'bit': 351,
         'yummy': 264,
         'leftovers': 96,
         'last': 176,
         'combining': 2,
         'me': 382,
         'calls': 19,
         'was': 2208,
         'instructions': 50,
         'whole': 199,
         'bought': 49,
         'super': 121,
         'taste': 380,
         'on': 1132,
         'for': 2997,
         'and': 3778,
         'wine': 67,
         'in': 1711,
         'lucky': 3,
         'over': 343,
         'is': 1591,
         'di